In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 1
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC50\ESC50_kFold_3s\..\TrainDataPreprocessing.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 131.94 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_3S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 2.7149057388305664 Train_acc: 11.270834%
Train Epoch: 2/90 Train_Loss: 2.0451598167419434 Train_acc: 28.750000%
Train Epoch: 3/90 Train_Loss: 1.7193294763565063 Train_acc: 43.333336%
Train Epoch: 4/90 Train_Loss: 1.5136773586273193 Train_acc: 54.583336%
Train Epoch: 5/90 Train_Loss: 0.9691511392593384 Train_acc: 64.541672%
Train Epoch: 6/90 Train_Loss: 0.8528762459754944 Train_acc: 72.250000%
Train Epoch: 7/90 Train_Loss: 0.871161162853241 Train_acc: 78.187500%
Train Epoch: 8/90 Train_Loss: 0.541697084903717 Train_acc: 81.854172%
Train Epoch: 9/90 Train_Loss: 0.6579334735870361 Train_acc: 87.375000%
Train Epoch: 10/90 Train_Loss: 0.3715301752090454 Train_acc: 88.520836%
Train Epoch: 11/90 Train_Loss: 0.372670441

Train Epoch: 16/90 Train_Loss: 0.19997823238372803 Train_acc: 95.312500%
Train Epoch: 17/90 Train_Loss: 0.11150502413511276 Train_acc: 96.187500%
Train Epoch: 18/90 Train_Loss: 0.16404187679290771 Train_acc: 96.291672%
Train Epoch: 19/90 Train_Loss: 0.18697091937065125 Train_acc: 96.333336%
Train Epoch: 20/90 Train_Loss: 0.09468022733926773 Train_acc: 96.395836%
Train Epoch: 21/90 Train_Loss: 0.11783937364816666 Train_acc: 96.479172%
Train Epoch: 22/90 Train_Loss: 0.12708966434001923 Train_acc: 96.541672%
Train Epoch: 23/90 Train_Loss: 0.04255615919828415 Train_acc: 96.687500%
Train Epoch: 24/90 Train_Loss: 0.10517852753400803 Train_acc: 96.645836%
Train Epoch: 25/90 Train_Loss: 0.22323957085609436 Train_acc: 96.937500%
Train Epoch: 26/90 Train_Loss: 0.1773623377084732 Train_acc: 96.708336%
Train Epoch: 27/90 Train_Loss: 0.10574749112129211 Train_acc: 96.687500%
Train Epoch: 28/90 Train_Loss: 0.0058736479841172695 Train_acc: 97.145836%
Train Epoch: 29/90 Train_Loss: 0.17699940502643585

Train Epoch: 33/90 Train_Loss: 0.10259565711021423 Train_acc: 97.354172%
Train Epoch: 34/90 Train_Loss: 0.0016545715043321252 Train_acc: 97.520836%
Train Epoch: 35/90 Train_Loss: 0.030785322189331055 Train_acc: 97.458336%
Train Epoch: 36/90 Train_Loss: 0.08463424444198608 Train_acc: 97.416672%
Train Epoch: 37/90 Train_Loss: 0.24550209939479828 Train_acc: 97.500000%
Train Epoch: 38/90 Train_Loss: 0.1648324429988861 Train_acc: 97.520836%
Train Epoch: 39/90 Train_Loss: 0.15348923206329346 Train_acc: 97.479172%
Train Epoch: 40/90 Train_Loss: 0.04921197518706322 Train_acc: 97.500000%
Train Epoch: 41/90 Train_Loss: 0.12155893445014954 Train_acc: 97.520836%
Train Epoch: 42/90 Train_Loss: 0.03523411229252815 Train_acc: 97.187500%
Train Epoch: 43/90 Train_Loss: 0.04426368698477745 Train_acc: 97.354172%
Train Epoch: 44/90 Train_Loss: 0.16599097847938538 Train_acc: 97.562500%
Train Epoch: 45/90 Train_Loss: 0.035970479249954224 Train_acc: 97.541672%
Train Epoch: 46/90 Train_Loss: 0.205621004104614

Train Epoch: 50/90 Train_Loss: 0.06295369565486908 Train_acc: 97.104172%
Train Epoch: 51/90 Train_Loss: 0.0369228757917881 Train_acc: 97.083336%
Train Epoch: 52/90 Train_Loss: 0.20022201538085938 Train_acc: 97.083336%
Train Epoch: 53/90 Train_Loss: 0.034370336681604385 Train_acc: 97.145836%
Train Epoch: 54/90 Train_Loss: 0.20802828669548035 Train_acc: 97.166672%
Train Epoch: 55/90 Train_Loss: 0.051289044320583344 Train_acc: 97.145836%
Train Epoch: 56/90 Train_Loss: 0.004200481344014406 Train_acc: 96.854172%
Train Epoch: 57/90 Train_Loss: 0.06659970432519913 Train_acc: 97.125000%
Train Epoch: 58/90 Train_Loss: 0.08551932871341705 Train_acc: 96.979172%
Train Epoch: 59/90 Train_Loss: 0.08121845871210098 Train_acc: 96.979172%
Train Epoch: 60/90 Train_Loss: 0.07561478018760681 Train_acc: 97.020836%
Train Epoch: 61/90 Train_Loss: 0.06725915521383286 Train_acc: 97.125000%
Train Epoch: 62/90 Train_Loss: 0.03753656521439552 Train_acc: 97.166672%
Train Epoch: 63/90 Train_Loss: 0.1028629019856453

Train Epoch: 68/90 Train_Loss: 0.03151554986834526 Train_acc: 97.437500%
Train Epoch: 69/90 Train_Loss: 0.13799917697906494 Train_acc: 97.395836%
Train Epoch: 70/90 Train_Loss: 0.08232541382312775 Train_acc: 97.354172%
Train Epoch: 71/90 Train_Loss: 0.12154701352119446 Train_acc: 97.416672%
Train Epoch: 72/90 Train_Loss: 0.06780315935611725 Train_acc: 97.416672%
Train Epoch: 73/90 Train_Loss: 0.06838048994541168 Train_acc: 97.437500%
Train Epoch: 74/90 Train_Loss: 0.11312359571456909 Train_acc: 97.437500%
Train Epoch: 75/90 Train_Loss: 0.0731852799654007 Train_acc: 97.437500%
Train Epoch: 76/90 Train_Loss: 0.07163576781749725 Train_acc: 97.437500%
Train Epoch: 77/90 Train_Loss: 0.03236333280801773 Train_acc: 97.437500%
Train Epoch: 78/90 Train_Loss: 0.04439600557088852 Train_acc: 97.437500%
Train Epoch: 79/90 Train_Loss: 0.15430448949337006 Train_acc: 97.437500%
Train Epoch: 80/90 Train_Loss: 0.06040739268064499 Train_acc: 97.437500%
Train Epoch: 81/90 Train_Loss: 0.12248954176902771 T

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()